## Prerequisites
### Standard imports

In [1]:
from __future__ import print_function, unicode_literals
import os
import json

### External libraries

To facilitate easier development, use existing libraries

[Requests-OAuthlib](https://requests-oauthlib.readthedocs.io)  
`pip install requests-oauthlib`  
An easy to use library for building OAuth1 and OAuth2 clients

[Tabulate](https://pypi.python.org/pypi/tabulate/)  
`pip install tabulate`  
Pretty-print tabular data

[Jupyter](http://jupyter.org)  
`pip install jupyter`  
This notebook is based on Jupyter (formerly IPython). Just using display to directly output tables

In [2]:
from requests_oauthlib import OAuth2Session
from tabulate import tabulate
from IPython import display

### Fix Python 2

In [3]:
try:
    input = raw_input
except NameError:
    pass

### Standard configuration

In [4]:
client_id = 'contestant'
client_secret = 'secret'
hostroot = 'http://api.beyondhackathon.com/'
scope = ['read', 'write']

### Custom configuration
This is the URL of your web application, where the OAuth code will end up

In [5]:
redirect_uri = 'http://example.com/auth_callback'

### Ignore the fact that the web server is only on HTTP

Normally, this would be done over HTTPS, but for the purposes of this hackathon, this will be done over a simple HTTP server.

In [6]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1' # Ignore any HTTPS errors (for example the ones generated by Fiddler)

### Web debugging proxy configuration

If you are using [Fiddler](http://www.getfiddler.com/) or any other web debugging proxy, uncomment the below lines to make OAuth2 work with the proxy.

In [7]:
# os.environ['http_proxy'] = 'http://localhost:8888' # default Fiddler port
# os.environ['https_proxy'] = 'http://localhost:8888'

## Get authorization URL

In [8]:
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, scope=scope)
authorization_url, state = oauth.authorization_url(hostroot + 'authorization/authorize')

## Visit the URL manually
You need to visit the URL from a browser and login as the end-user

In a real web application, you can simply redirect (302) the user

In [9]:
print('Please go to %s and authorize access.' % authorization_url)

Please go to http://api.beyondhackathon.com/authorization/authorize?response_type=code&client_id=contestant&redirect_uri=http%3A%2F%2Fexample.com%2Fauth_callback&scope=read+write&state=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx and authorize access.


## Manually enter the result URL
Since this is just a notebook and not a real web site, pretend you opened the authorize callback page. Just paste the URL that your browser has displayed.

In [10]:
authorization_response = input('Enter the full callback URL:')

Enter the full callback URL:http://example.com/auth_callback?code=xxxxxx&state=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


## User token for authorization

In [11]:
oauth_url = hostroot + 'authorization/token'
token = oauth.fetch_token(oauth_url, client_secret=client_secret, authorization_response=authorization_response)

## You are ready to use the API

## Retrieve customer info

In [12]:
customer_info_request = oauth.get(hostroot + 'customers/me/info')
me = customer_info_request.json()

In [13]:
print(me)

{'first_name': 'Aryana', 'email_addresses': ['aframi@example.com'], 'mobile_numbers': ['+306919045271'], 'last_name': 'Frami', 'corporate': False}


## Retrieve customer unread messages

In [14]:
message_count_request = oauth.get(hostroot + '/customers/me/messages-count')
message_count = message_count_request.json()['number']

In [15]:
print(message_count)

1


## Get list of product (i.e. cards and accounts)

In [16]:
products = oauth.get(hostroot + '/customers/me/products').json()

## Generate list of statements

In [17]:
statements = []
for p in products:
    if p['type'] == 'account':
        r3 = oauth.get(hostroot + '/accounts/{}/statement'.format(p['contract_number']))
    elif p['type'] == 'card':
        r3 = oauth.get(hostroot + '/cards/{}/statement'.format(p['contract_number']))
    for sl in r3.json():
        sl['type'] = p['type']
        sl['alias'] = p['alias']
        statements.append(sl)

## List statements as a table

Since they are dictionaries, the order is not guaranteed

In [18]:
display.Markdown(tabulate(statements, tablefmt='pipe', headers="keys", floatfmt=".2f"))

| type    | alias     |   amount | value_date   | description                                  |   new_balance | transaction_date   |
|:--------|:----------|---------:|:-------------|:---------------------------------------------|--------------:|:-------------------|
| account | Bisque    |    20.69 | 2016-01-06   | Κατάθεση 648                                 |       9727.69 | 2016-01-06         |
| account | Bisque    |   -75.81 | 2016-01-06   | Ανάληψη 238                                  |       9651.88 | 2016-01-06         |
| account | Bisque    |   -76.37 | 2016-01-07   | Πληρωμή Βεβαιωμένης Οφειλής 740              |       9575.51 | 2016-01-07         |
| account | Bisque    |   -73.16 | 2016-01-07   | Πληρωμή Βεβαιωμένης Οφειλής 002              |       9502.35 | 2016-01-07         |
| account | Bisque    |   -96.73 | 2016-01-07   | Ανάληψη 829                                  |       9405.62 | 2016-01-07         |
| account | Bisque    |    13.01 | 2016-01-07   | Κατάθεση 146                                 |       9418.63 | 2016-01-07         |
| account | Bisque    |   -39.67 | 2016-01-07   | Πληρωμή Βεβαιωμένης Οφειλής 780              |       9378.96 | 2016-01-07         |
| account | DimGray   |   -88.88 | 2016-04-22   | Πληρωμή Βεβαιωμένης Οφειλής 029              |       8703.12 | 2016-04-22         |
| account | DimGray   |    90.35 | 2016-04-22   | Κατάθεση 356                                 |       8793.47 | 2016-04-22         |
| account | Plum      |     1.62 | 2016-01-11   | Κατάθεση 448                                 |       4691.62 | 2016-01-11         |
| account | Plum      |    57.91 | 2016-01-11   | Κατάθεση 859                                 |       4749.53 | 2016-01-11         |
| card    | LightGray |   173.25 | 2016-01-25   | Αγορά Distributed optimal approach           |        173.25 | 2016-01-25         |
| card    | LightGray |    13.52 | 2016-01-26   | Αγορά Stand-alone bi-directional time-frame  |        186.77 | 2016-01-26         |
| card    | LightGray |   167.00 | 2016-01-26   | Αγορά Reactive 24/7 alliance                 |        353.77 | 2016-01-26         |
| card    | DeepPink  |   114.74 | 2016-01-18   | Αγορά Future-proofed scalable implementation |        114.74 | 2016-01-18         |